In [12]:
import torch
from torch import nn
from torchvision import models, transforms, datasets
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

In [13]:
# Define the model architecture
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = models.densenet121(pretrained=True)
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(1024, 1024)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.gap(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [14]:
# Set device to GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [15]:
# Create the model
model = CustomModel()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)


In [17]:
# Define the data transformations
image_size= (224, 224)
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

In [18]:
# Load the datasets
image_datasets = {
    'train': datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/NewData/data/train', data_transforms['train']),
    'val': datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/NewData/data/validation', data_transforms['val']),
    'test': datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/NewData/data/test', data_transforms['test'])
}

In [19]:
# Define the dataloaders
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=64, shuffle=True),
    'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=64),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=64)
}

In [20]:
# Train the model
model.to(device)
train_losses=[]
train_accs = []
for epoch in range(100):
    correct_train = 0
    total_train = 0
    running_loss=0.0

    for i, (inputs, labels) in enumerate(dataloaders['train'], 0):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        running_loss += loss.item()


    train_loss = running_loss/len(dataloaders['train'])
    train_losses.append(train_loss)
    train_acc = (correct_train/total_train)*100
    train_accs.append(train_acc)
    print(f'Epoch: {epoch + 1}, Loss: {running_loss:.4f}, Accuracy: {train_acc:.4f}')




Epoch: 1, Loss: 43.8695, Accuracy: 9.4527
Epoch: 2, Loss: 43.0341, Accuracy: 14.8425
Epoch: 3, Loss: 42.0813, Accuracy: 21.7247
Epoch: 4, Loss: 41.0804, Accuracy: 30.7629
Epoch: 5, Loss: 40.2016, Accuracy: 39.3864
Epoch: 6, Loss: 39.2820, Accuracy: 48.6733
Epoch: 7, Loss: 38.3456, Accuracy: 56.1360
Epoch: 8, Loss: 37.4548, Accuracy: 62.7695
Epoch: 9, Loss: 36.5954, Accuracy: 67.9104
Epoch: 10, Loss: 35.6491, Accuracy: 70.6468
Epoch: 11, Loss: 34.7076, Accuracy: 74.2123
Epoch: 12, Loss: 33.6874, Accuracy: 77.4461
Epoch: 13, Loss: 32.8301, Accuracy: 78.7728
Epoch: 14, Loss: 31.8744, Accuracy: 80.8458
Epoch: 15, Loss: 30.8721, Accuracy: 82.0896
Epoch: 16, Loss: 29.9175, Accuracy: 81.9237
Epoch: 17, Loss: 29.0028, Accuracy: 83.9138
Epoch: 18, Loss: 28.0157, Accuracy: 84.9917
Epoch: 19, Loss: 27.0679, Accuracy: 85.1575
Epoch: 20, Loss: 25.9962, Accuracy: 86.6501
Epoch: 21, Loss: 25.1183, Accuracy: 86.8988
Epoch: 22, Loss: 24.1766, Accuracy: 87.2305
Epoch: 23, Loss: 23.3891, Accuracy: 87.230

In [21]:
# Evaluate the model on test data
valid_losses = []
valid_accs = []
model.eval()
with torch.no_grad():
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in dataloaders['val']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    valid_loss = running_loss/len(dataloaders['val'])
    valid_losses.append(valid_loss)
    valid_acc =(correct / total) *100
    valid_accs.append(valid_acc)
print(f'Test accuracy: {(correct / total)*100}')

Test accuracy: 94.04761904761905


In [22]:
# Generate classification report
model.eval()
correct_1 = 0
correct_5 = 0
y_true = []
y_pred = []
total = 0
with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)

        _, predicted = outputs.topk(5, 1, True, True)
        predicted = predicted.t()
        correct = predicted.eq(labels.view(1, -1).expand_as(predicted))
        correct_1 += correct[:1].reshape(-1).float().sum(0)
        correct_5 += correct[:5].reshape(-1).float().sum(0)
        total += labels.size(0)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(pred.cpu().numpy())
top1_error_rate = 1 - (correct_1/total).cpu().numpy()
top5_error_rate = 1 - (correct_5/total).cpu().numpy()
print('top1_error_rate: {:.2f}%'.format(top1_error_rate*100))
print('top5_error_rate: {:.2f}%'.format(top5_error_rate*100))


class_names = image_datasets['train'].classes
classification_rep = classification_report(y_true, y_pred, target_names=class_names)
print('Classification Report:')
print(classification_rep)


top1_error_rate: 4.44%
top5_error_rate: 0.37%
Classification Report:
              precision    recall  f1-score   support

         151       0.97      1.00      0.98        30
         152       0.93      0.93      0.93        30
         153       1.00      0.90      0.95        30
         154       1.00      0.97      0.98        30
         155       0.91      0.97      0.94        30
         157       0.90      0.90      0.90        30
         159       0.97      1.00      0.98        30
         160       0.94      1.00      0.97        30
         161       1.00      0.93      0.97        30

    accuracy                           0.96       270
   macro avg       0.96      0.96      0.96       270
weighted avg       0.96      0.96      0.96       270

